* [ex1](#ex1.py)
* [ex2](#ex2.py)
* [ex3](#ex3.py)
* [ex4](#ex4)

# ex1.py

Your web server should listen on ` 127.0.0.1:5000 `. It should expect POST queries to the
URL ` /hw2/ex1 `. The POST content is a JSON of the following form:
{"user": user,"pass": password}

If the password is correct, the response must have the status code  200 . If it is not correct, it
should be  400 . Your verification function must use the same algorithm as the one used in the
first exercise of the first homework (` com402.epfl.ch/hw1/ex1 `).

The library you must use is named Flask. You can find all the documentation at
http://flask.pocoo.org/docs/0.12/ . You must use Python3 and submit only one file  `ex1.py `

To get the token, you can upload your script to `com402.epfl.ch/hw2/` for exercise 1.
For testing, your web-server can serve a simple html file, e.g., Hello World! Note that our
verification scripts won’t look at the HTML that you send back.

In [ ]:
[...]

# ex2.py
This exercise is about implementing a password-authenticated key agreement protocol,
PAKE​ . PAKE serves two main purposes: derivation of a cryptographically secure shared key
from a low entropy password and proving the knowledge of a secret password to each other
without actual transmission of this password. This is useful for a lot of applications. For
example, ProtonMail is an end-to-end encrypted email service provider which uses a PAKE
protocol named Secure Remote Password protocol (SRP).

You have to implement the client part of the SRP protocol that interacts with our server using
websockets at ` ws://com402.epfl.ch/hw2/ws​ `. This client should perform the SRP exchange
with  the username being your EPFL EMAIL address and with the password that has been
generated in the previous exercise.

The web server implements a simplified version of the ​ SRP protocol described in the RFC: https://www.ietf.org/rfc/rfc2945.txt

At the end of the protocol, both parties should have the same shared secret  S  .
To get the token, send ` H(A || B || S) ` to the server at the end, and look at the response!

#### Network
All communication happens through a websocket channel. You can create this channel by
connecting to ` ws://com402.epfl.ch/hw2/ws `.
#### Implementation
You can
use  this library: https://websockets.readthedocs.io/en/stable/

Creating a websocket client is very easy and you can just use  this snippet: https://websockets.readthedocs.io/en/stable/intro.html

# ex3.py
In this exercise you are asked to create a web-server that serves cookies to clients based on
whether they are simple users or administrators of the system. However, as you now know,
malicious users might try to tamper with the cookie to get administrative access.
You decided to protect the authenticity of your cookies, so you are going to add a
Keyed-Hash Message Authentication Code (HMAC) as an additional field. The secret key of
the HMAC is  your password  of hw1/ex1 encoded in utf8.

You must provide a single python3 script  ex3.py implementing a web server using the Flask
library that listens on ` 127.0.0.1:5000 `.

To login, POST a json at ​ localhost:5000/ex3/login:

`{"user": user,"pass": password}`

The response must contain a cookie. It must be an administrator cookie if the username is
“administrator” and the password is “42”. Otherwise, it must be a regular cookie. The name of the cookie is ` LoginCookie`

The cookie should have the same fields as in the exercise 2 of homework 1 with an
additional field holding an HMAC. Examples of a cookie:

administrator,1489662453,com402,hw2,ex3,administrator,HMAC

Yoda,1563905697,com402,hw2,ex3,user,HMAC



Upon a POST request, ` localhost:5000/ex3/list` must return a specific HTTP status
* code 200 if the user is the administrator
* code 201 if the user is a simple user
* code 403 if the cookie is tampered

In [ ]:
# This version does not use 'redirect'

import base64
import json
import hmac
#from flask import after_this_request
from flask import current_app
from flask import Flask
from flask import request
#from flask import redirect
from flask import make_response

# Implement the verification function
def superencryption(msg):
    key = "Never send a human to do a machine's job"
    if (len(key) < len(msg)):
        key = key + key[0:len(msg) - len(key)]
    amsg = map(ord, msg)
    akey = map(ord, key[0:len(msg)])
    y = ''.join([chr(v ^ i) for (i,v) in zip(akey, amsg)])
    return base64.b64encode(y.encode('utf-8'))

key = superencryption('lucia.monterosanchis@epfl.ch')

def check_pwd(username, password):
    return (username=='administrator' and str(password)=='42')

def compute_cookie(username, password, type_):
    fields = ",".join([username, '1489662453', 'com402', 'hw2', 'ex3', type_])
    hmac_obj = hmac.new(key, fields.encode('utf-8'))
    cookie = ",".join([fields, hmac_obj.hexdigest()])
    return base64.b64encode(cookie.encode('utf-8'))

def check_cookie(cookie_64):
    cookie = base64.b64decode(cookie_64)
    split_cookie = cookie.decode('utf-8').split(",")
    fields = ",".join(split_cookie[:6])
    type_ = split_cookie[5]
    hmac_rec = split_cookie[6]
    
    h = hmac.new(key, fields.encode('utf-8'))
    cookie_ok = hmac.compare_digest(hmac_rec, h.hexdigest())
    is_admin = (type_ == 'administrator')
    
    return cookie_ok, is_admin

# Implement the Flask server
app = Flask(__name__) # this is used when using a single module

@app.route('/ex3/login', methods=['POST'])
def login_app():
    content = request.get_json()
    username = content['user']
    password = content['pass']
    is_admin = check_pwd(username, password)
    if is_admin:
        cookie = compute_cookie(username, password, 'administrator')
    else:
        cookie = compute_cookie(username, password, 'user')

    response = make_response()  
    response.set_cookie('LoginCookie', cookie)
    return response

@app.route('/ex3/list', methods=['POST'])
def list_app():
    cookie = request.cookies.get('LoginCookie')
    cookie_ok, is_admin = check_cookie(cookie)
    if (cookie_ok and is_admin):
        return ':D', 200
    elif cookie_ok:
        return ':)', 201
    else:
        return ':(', 403

# run the Flask app
if __name__ == "__main__":
    app.run()